# Scraping artivles from Huffingtonpost website

In [14]:
#I import the necessary packages to webscrape
import requests
from lxml.html import fromstring
import time
import pandas as pd

#I create a function to extract the information from the website
def fetch_article_html(url):
    response = requests.get(url)
    return response.text

#I create a function to extract the text of the article and to put together all of the paragraphs
def extract_article_text(html):
    tree = fromstring(html)
    article_text = tree.xpath("//div[1]/p/text() | //div[1]/p/a/text()")
    return ' '.join(article_text)

#I define the url of the website and loop over the 50 pages present in the section I am interested in
MAXPAGES = 50
baseurl = "https://www.huffingtonpost.it/argomenti/elezioni_2022/"
allurls = [f"{baseurl}{i}/" for i in range(1, MAXPAGES + 1)]

#I create a list in which to store the information
articles_data = []

#I loop over every url and check if the page exist. Then I extract the link and title of each article from the website and 
#then for every link (so for every aerticle) I retrieve the test of that article. 
for url in allurls:
    try:
        #I check if the page exists
        response = requests.head(url)
        if response.status_code == 404:
            print("Page not found:", url)
            continue

        #I fetch the HTML source from the website
        response = requests.get(url)
        htmlsource = response.text

        tree = fromstring(htmlsource)

        #I find the links and titles
        article_links = tree.xpath("//article/div/h2/a/@href")
        article_title = tree.xpath("//article/div/h2/a/text()")

        #I extract texts
        for link, title in zip(article_links, article_title):
            try:
                #I fetch the HTML source of each article page
                article_html = fetch_article_html(link)

                #I extract the full article text
                article_text = extract_article_text(article_html)

                #I append the links, titles and text to the list created above
                articles_data.append({
                    'Link': link,
                    'Title': title,
                    'Text': article_text
                })

                #I delay of 1 second before the next request
                time.sleep(1) 

            except requests.exceptions.RequestException as e:
                print("Connection error:", e)
                continue

    except requests.exceptions.RequestException as e:
        print("Connection error:", e)
        continue

# I create a dataframe from the articles data dictionary
articles_Huff = pd.DataFrame.from_dict(articles_data)
articles_Huff['News outlet'] = pd.Series('HuffPost', index=articles_Huff.index)
articles_Huff.to_csv('articles_Huff.csv', index=False)

# I print the DataFrame
articles_Huff.head()

,Link,Title,Text,News outlet
0,https://www.huffingtonpost.it/blog/2022/10/26/...,Il Partito Democratico torni ad essere un part...,Si Ã¨ insediato il governo piÃ¹ a destra del...,HuffPost
1,https://www.huffingtonpost.it/politica/2022/10...,"Neo-ruinismo al potere. Roccella, Mantovano e ...","La svolta conservatrice italiana, sullâimpro...",HuffPost
2,https://www.huffingtonpost.it/politica/2022/10...,Disuniti al Quirinale. Le tre opposizioni non ...,"Le opposizioni arrivano in ordine sparso, fort...",HuffPost
3,https://www.huffingtonpost.it/politica/2022/10...,"Baldassarre, Mennuni e non solo. C'è un grumo ...","âGrazie Mateusz Morawiecki, difendiamo insie...",HuffPost
4,https://www.huffingtonpost.it/politica/2022/10...,Berlusconi manda affan... La Russa in diretta Tv,Un acceso diverbio tra Ignazio La Russa e Sil...,HuffPost


I create a condition with a list of rows and then use it to extract those rows from the dataset and put into another dataset, 
#which will serve for the labeled dataset. Then, I drop those columns from the original dataset, as this is the unlabeled dataset. I save both dataframes and I re-set their indexes.

In [15]:
condition = list(range(1, 29)) + list(range(320, 454)) + list(range(962, 990))
articlesHuff_labeled = articles_Huff.loc[condition].copy()
articlesHuff_labeled.head()
articlesHuff_labeled.to_csv('articlesHuff_labeled.csv', index=False)
articlesHuff_unlabeled = articles_Huff.drop(list(range(1, 29)) + list(range(320, 454)) + list(range(962, 990)))
articlesHuff_unlabeled.to_csv('articlesHuff_unlabeled.csv', index=False)
#I restore the indexes of both dataframes
articlesHuff_labeled.reset_index(drop=True, inplace=True)
articlesHuff_unlabeled.reset_index(drop=True, inplace=True)

In [16]:
#I check that everything went smoothly
articlesHuff_labeled

,Link,Title,Text,News outlet
0,https://www.huffingtonpost.it/politica/2022/10...,"Neo-ruinismo al potere. Roccella, Mantovano e ...","La svolta conservatrice italiana, sullâimpro...",HuffPost
1,https://www.huffingtonpost.it/politica/2022/10...,Disuniti al Quirinale. Le tre opposizioni non ...,"Le opposizioni arrivano in ordine sparso, fort...",HuffPost
2,https://www.huffingtonpost.it/politica/2022/10...,"Baldassarre, Mennuni e non solo. C'è un grumo ...","âGrazie Mateusz Morawiecki, difendiamo insie...",HuffPost
3,https://www.huffingtonpost.it/politica/2022/10...,Berlusconi manda affan... La Russa in diretta Tv,Un acceso diverbio tra Ignazio La Russa e Sil...,HuffPost
4,https://www.huffingtonpost.it/politica/2022/10...,"Cerchietto gioiello per Fascina, total black p...",Domina la sobrietÃ negli outfit della prima ...,HuffPost
...,...,...,...,...
185,https://www.huffingtonpost.it/blog/2022/08/06/...,Gli europeisti di oggi sono gli antieuropeisti...,"In campagna elettorale, purtroppo, si rischia ...",HuffPost
186,https://www.huffingtonpost.it/blog/2022/08/06/...,Che le elezioni del 25 settembre non siano un ...,Nella Terra siamo sicuramente a un bivio: uno ...,HuffPost
187,https://www.huffingtonpost.it/blog/2022/08/06/...,Ora tutte le forze politiche si trovano a prop...,Dopo la traumatica - e a mio parere irresponsa...,HuffPost
188,https://www.huffingtonpost.it/politica/2022/08...,Letta e il cubo di Rubik. In un modo o nell'al...,"il co-portavce dei Verdi Angelo Bonelli, mentr...",HuffPost


In [17]:
#I check that everything went smoothly
articlesHuff_unlabeled

,Link,Title,Text,News outlet
0,https://www.huffingtonpost.it/blog/2022/10/26/...,Il Partito Democratico torni ad essere un part...,Si Ã¨ insediato il governo piÃ¹ a destra del...,HuffPost
1,https://www.huffingtonpost.it/blog/2022/10/02/...,Se il Pd vuole rinascere abbandoni l'arcaica p...,Mettiamoci nei panni di quei milioni di cittad...,HuffPost
2,https://www.huffingtonpost.it/economia/2022/10...,Un Figliuolo del gas. Meloni vuole centralizza...,Nei contatti che Giorgia Meloni ha allargato n...,HuffPost
3,https://www.huffingtonpost.it/politica/2022/10...,Bossi torna in campo: nasce il Comitato Nord. ...,Nei giorni della battaglia piÃ¹ importante per...,HuffPost
4,https://www.huffingtonpost.it/politica/2022/10...,Niente veti. Berlusconi vede Meloni e dà un ai...,âLa nostra rosa la componiamo noi e non mett...,HuffPost
...,...,...,...,...
805,https://www.huffingtonpost.it/politica/2022/08...,"Meloni, Conte, Salvini, Calenda &Co.; Le pagel...","Makkox, il fumettista di Propaganda Live, inte...",HuffPost
806,https://www.huffingtonpost.it/politica/2022/08...,"Mario Tronti: ""Non vedo il rischio di rigurgit...","Mario Tronti, filosofo , una vita a sinistra, ...",HuffPost
807,https://www.huffingtonpost.it/politica/2022/08...,"Bonelli dice sì, Fratoianni sì però. Palla all...",Ancora una giornata di incertezza. Ancora una ...,HuffPost
808,https://www.huffingtonpost.it/politica/2022/08...,Accrocchio poligamo. Letta non riesce a teners...,"In fondo, Ã¨ semplice. Carlo Calenda â lo ha...",HuffPost
